# MNIST digits classification using logistic regression

## AUTHOR: Rajesh Bondugula

**This notebook implements classification of MNIST digits using one-vs-all logistic regression.**

In [2]:
# Import numpy and matplotlib modiules.
import scipy.special as scp_spl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fmin_bfgs
from loader import loadmnist
%matplotlib inline

from IPython.core.display import HTML
pd.set_option('display.max_rows', 10)

css ="""
body {
    margin: 0;
    font-family: Helvetica;
}
table.dataframe {
    border-collapse: collapse;
    border: none;
}
table.dataframe tr {
    border: none;
}
table.dataframe td, table.dataframe th {
    margin: 0;
    border: 1px solid white;
    padding-left: 0.25em;
    padding-right: 0.25em;
}
table.dataframe th:not(:empty) {
    background-color: #fec;
    text-align: left;
    font-weight: normal;
}
table.dataframe tr:nth-child(2) th:empty {
    border-left: none;
    border-right: 1px dashed #888;
}
table.dataframe td {
    border: 2px solid #ccf;
    background-color: #f4f4ff;
}"""

HTML('<style>{}</style>'.format(css))


## 1. Load the data set

- Download the data set from [http://yann.lecun.com/exdb/mnist/](http://yann.lecun.com/exdb/mnist/)
- Load the data using loadmnist() from loader.py

In [3]:
# Load the MNIST data.
X_train, y_train = loadmnist('train-images.idx3-ubyte', 'train-labels.idx1-ubyte')
X_test, y_test = loadmnist('t10k-images.idx3-ubyte', 't10k-labels.idx1-ubyte')

## 2. Implement the cost function

$$
J\left(\theta\right) = \frac{1}{m}\sum_{i=1}^{m} \left[-y^{(i)}log\left(h_{\theta}\left(x^{(i)}\right)\right) -\left(1-y^{(i)}\right)log\left(1-h_{\theta}\left(x^{(i)}\right)\right) \right] \\ x^{\left(i\right)} is \: the \: i^{th} sample \: of \:x
$$

In [4]:
def calculate_cost(theta, X, y, lamba_reg=0):
    """The function returns the cost of the logistic regression

       X     : Input array mxn. m rows are number of samples and n features for each sample.
       y     : Original class of the input
       theta : training parameters
    """
    # Get number of rows
    m = X.shape[0]

    # Convert y and theta into vectors.
    y = np.array(y).reshape(len(y), 1)
    theta = np.array(theta).reshape(len(theta), 1)

    # Append ones as first column to input matrix X.
    # This is because theta is (n + 1) x 1 due to the intercept.
    # The feature value for intercept is 1.
    if len(theta) - X.shape[1] == 1:
        X = np.column_stack([np.ones(m), X])

    # Calculate the h(x)
    h = scp_spl.expit(np.dot(X, theta))

    # Due the precision constraints, value close to 1 is rounded to 1 and hence log(1-h) is not defined.
    # We adjust the presision so that log is valid.
    # We pick a small value for which precision is retained. exp(-37) is found by trial and error method.
    # We make sure that none of the values neither 0 nor 1.
    precisoin_adjust_value = np.exp(-37)

    h = np.where(h == 1, 1 - precisoin_adjust_value, h)
    h = np.where(h == 0, precisoin_adjust_value, h)

    # Calculate the cost
    J = sum(-y * np.log(h) - (1 - y) * np.log(1 - h)) / m
    
    # Adjust the cost to add the regularization term
    reg_value = lamba_reg/(2.0 * m) * sum(np.where(np.arange(len(theta)) == 0, 0, theta.flatten()**2))
    return J[0] + reg_value

## 3. Implement the gradient function

$$
    \frac{\partial \: J(\theta)}{\partial \theta _j} = \frac{1}{m}\sum_{i = 1}^{m} \left( h_\theta \left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)}
$$

$$
    \theta_j := \theta_j - \alpha \frac{\partial \: J(\theta)}{\partial \theta _j}
$$

$$
    h_{\theta}\left(x\right)=\frac{1}{1 + e^{-\theta^{T}x}}
$$

In [5]:
def calculate_gradient(theta, X, y, lambda_reg=0):
    """Calculate the gradient of cost function at theta"""

    # Get number of rows
    m = X.shape[0]

    # Convert y and theta into vectors.
    y = np.array(y).reshape(len(y), 1)
    theta = np.array(theta).reshape(len(theta), 1)

    # Append ones as first column to input matrix X.
    # This is because theta is (n + 1) x 1 due to the intercept.
    # The feature value for intercept is 1.
    if len(theta) - X.shape[1] == 1:
        X = np.column_stack([np.ones(m), X])

    # Calculate the h(x)
    h_y = scp_spl.expit(np.dot(X, theta)) - y
    grad = sum(h_y.repeat(X.shape[1], axis=1) * X)/ m
    
    # Add the regularization term
    grad += float(lambda_reg) * np.where(np.arange(len(theta)) == 0, 0, theta.flatten()) / m

    return grad

## 4. Perfom BFGS on training data to find optimum model parameters

In [17]:
# perform BFGS and find the parameters that minimize the function.
def get_model_parameters(X, y, param_size=3, lambda_reg=0):
    def f(theta):
        """"""
        return calculate_cost(theta, X, y, lambda_reg)

    def fprime(theta):
        return calculate_gradient(theta, X, y, lambda_reg)

    # Initial theta assigned to 0
    initial_theta = np.zeros(param_size)

    return fmin_bfgs(f, initial_theta, fprime, disp=True, maxiter=100)

## 5. Get the model parameters

- We get the model parameters using BFGS
- The parameters we get are in one-vs-all fashion.
- We calculate the probability of one class (label 1) by treating all other classes are with label 0.
- We calculate this for all classes.

In [31]:
for c in range(10):
    if c:
        tuned_theta = np.column_stack([tuned_theta, get_model_parameters(X_train, y_train == c, X_train.shape[1] + 1)])
    else:
        tuned_theta = get_model_parameters(X_train, y_train == c, X_train.shape[1] + 1)

         Current function value: -32.931025
         Iterations: 1
         Function evaluations: 25
         Gradient evaluations: 13
         Current function value: -32.390375
         Iterations: 1
         Function evaluations: 25
         Gradient evaluations: 13
         Current function value: -32.900540
         Iterations: 1
         Function evaluations: 25
         Gradient evaluations: 13
         Current function value: -32.796340
         Iterations: 1
         Function evaluations: 25
         Gradient evaluations: 13
         Current function value: -32.958668
         Iterations: 1
         Function evaluations: 25
         Gradient evaluations: 13
         Current function value: -33.214043
         Iterations: 1
         Function evaluations: 25
         Gradient evaluations: 13
         Current function value: -32.921717
         Iterations: 1
         Function evaluations: 25
         Gradient evaluations: 13
         Current function value: -32.700119
         It

C:\Users\rajeshb\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:34: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


## 6. Classify the digits

$$
Classify \: x \: in \: class \: Argmax\: i \: in: \: \theta _i ^Tx \\
i \: is \: the \: class \\
\theta \: is \: the \: matrix \: with \: model \: parameters \\
Each \: column \: of \: \theta \: represents \: parameters \: of \: class
$$

In [34]:
def predict(X, theta):

    # Get number of rows
    m = X.shape[0]

    # Convert theta into vectors if it is an array.
    if len(theta.shape) == 1:
        theta = np.array(theta).reshape(len(theta), 1)

    # Append ones as first column to input matrix X.
    # This is because theta is (n + 1) x 1 due to the intercept.
    # The feature value for intercept is 1.
    if len(theta) - X.shape[1] == 1:
        X = np.column_stack([np.ones(m), X])
    
    # return the class which has maximum probability.
    return np.dot(X, theta).argmax(axis=1).flatten()

## 7. Validate the model on training data

In [32]:
predicted_y = predict(X_train, tuned_theta)
print "Prediction score = ", sum(predicted_y == y_train)*100/float(len(y_train)), "%"

Prediction score =  71.475 %


## 8. Validate the model on testing data

In [33]:
predicted_y = predict(X_test, tuned_theta)
print "Prediction score = ", sum(predicted_y == y_test)*100/float(len(y_test)), "%"

Prediction score =  72.4 %
